In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from google.colab import drive

In [3]:
drive.mount('/content/drive/')
%cd /content/drive/My Drive/PG/

Mounted at /content/drive/
/content/drive/My Drive/PG


In [4]:
model_id = "tiiuae/falcon-rw-1b"
lora_adapter_path = "/content/drive/My Drive/PG/lora-falcon-de"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id, trust_remote_code=True
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [6]:
model = PeftModel.from_pretrained(
    base_model,
    lora_adapter_path,
    device_map="auto"
)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(50304, 2048)
        (h): ModuleList(
          (0-23): 24 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (query_key_value): lora.Linear(
                (base_layer): FalconLinear(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
           

In [7]:
def generate_german(prompt: str, max_new_tokens=128, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
prompt = "Was sollte ich den deutschen Bundeskanzler fragen, wenn ich ihn treffe"
print(generate_german(prompt))

Was sollte ich den deutschen Bundeskanzler fragen, wenn ich ihn treffe von der deutschen Bundesregierung eine eigene Antwort habe, wie sie wird wirksam haben, dass die deutsche Einheit im Gegenteil sein.
Ich mache es schwer, dass deutsche Einheit nur von ihnen verhindert.
Bitte stellen Sie sich an, wenn sie einen deutschen Kommanditenantwort haben.
Ich mache ein deutsches Kommandit durch den deut
